## Code for cs5785 final, Team Ground Truth
### 1. Feature Extraction part 
- Author: Kai Zhang kz298@cornell.edu, Hongliang CHI, hc962@cornell.edu
- If you need to access to the processed data directly, instead of generating them by running this code:
- Please read the README file and download the generated feature data from https://drive.google.com/drive/folders/1pkXLFcvuEkC_VQ-QunVUC9EjjlvF8-j5

In [5]:
import torch
import torch.utils.data
import numpy as np
import pandas as pd
import os
import imageio


########################################################################################################################
# data loader
########################################################################################################################

class ImgFolder(object):
    def __init__(self, img_folder):
        self.img_folder = img_folder
        self.cnt = len(os.listdir(self.img_folder))

    def __len__(self):
        return self.cnt

    def __getitem__(self, index):
        img = imageio.imread(os.path.join(self.img_folder, '{}.jpg'.format(index)))
        img = np.float32(img) / 255.0

        if len(img.shape) != 3:
            img = np.tile(img[:, :, np.newaxis], (1, 1, 3))

        __imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                            'std': [0.229, 0.224, 0.225]}
        for i in range(3):
            img[:, :, i] = (img[:, :, i] - __imagenet_stats['mean'][i]) / __imagenet_stats['std'][i]

        img = torch.from_numpy(np.ascontiguousarray(img.transpose(2, 0, 1))).contiguous().float()
        return img


def create_img_loader(batch_size):
    train_img_folder = '/images_train'
    train_img_data = ImgFolder(train_img_folder)
    num_workers = 4
    train_img_loader = torch.utils.data.DataLoader(train_img_data,
                                                   batch_size=batch_size, shuffle=False, num_workers=num_workers)

    test_img_folder = '/images_test'
    test_img_data = ImgFolder(test_img_folder)
    num_workers = 4
    test_img_loader = torch.utils.data.DataLoader(test_img_data,
                                                  batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_img_loader, test_img_loader


if __name__ == '__main__':
    import torchvision.models
    import torch.nn as nn

    out_dir = 'resnet_features'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    resnet_id = '_ResNext'
    model = torchvision.models.resnext101_32x8d(pretrained=True)

    device = 'cuda:0'
    model = model.to(device)
    model.eval()

    modules = list(model.children())
    prev_layers = nn.Sequential(*modules[:-1])
    last_layer = modules[-1]

    train_img_loader, test_img_loader = create_img_loader(100)

    # embed train img
    total_cnt = len(train_img_loader.dataset)
    train_img_features = np.zeros((total_cnt, 1000+2048), dtype=np.float32)
    cnt = 0
    for batch_idx, item in enumerate(train_img_loader):
        input = item.to(device)
        with torch.no_grad():
            prev_out = prev_layers(input)
            prev_out = prev_out.squeeze(3)
            prev_out = prev_out.squeeze(2)

            out = last_layer(prev_out)

            prev_out = prev_out.cpu().numpy()
            out = out.cpu().numpy()

        train_img_features[cnt:cnt+item.shape[0], :1000] = out
        train_img_features[cnt:cnt+item.shape[0], 1000:] = prev_out
        cnt += item.shape[0]

    print('train shape: {}'.format(train_img_features.shape))
    pd.DataFrame(data=train_img_features).to_csv(os.path.join(out_dir, 'resnet{}_train.csv'.format(resnet_id)))

    # embed test img
    total_cnt = len(test_img_loader.dataset)
    test_img_features = np.zeros((total_cnt, 1000+2048), dtype=np.float32)
    cnt = 0
    for batch_idx, item in enumerate(test_img_loader):
        input = item.to(device)
        with torch.no_grad():
            prev_out = prev_layers(input)
            prev_out = prev_out.squeeze(3)
            prev_out = prev_out.squeeze(2)

            out = last_layer(prev_out)

            prev_out = prev_out.cpu().numpy()
            out = out.cpu().numpy()

        test_img_features[cnt:cnt+item.shape[0], :1000] = out
        test_img_features[cnt:cnt+item.shape[0], 1000:] = prev_out
        cnt += item.shape[0]

    print('test shape: {}'.format(test_img_features.shape))
    pd.DataFrame(data=test_img_features).to_csv(os.path.join(out_dir, 'resnet{}_test.csv'.format(resnet_id)))
